# Laporan Proyek Machine Learning - SistemRekomendasi

## Domain Proyek

Pada proyek ini, kita membahas masalah pengelolaan sampah dan memberikan solusi berbasis machine learning untuk memberikan rekomendasi pengelolaan sampah yang lebih tepat. Pengelolaan sampah yang efisien sangat penting untuk mengurangi dampak negatif terhadap lingkungan, terutama dengan meningkatnya volume sampah. Pentingnya pengelolaan sampah yang tepat tidak hanya membantu mengurangi pencemaran tetapi juga mengurangi penggunaan sumber daya alam. Oleh karena itu, pengelolaan yang efisien sangat dibutuhkan agar dapat mengoptimalkan proses daur ulang, komposting, dan pembuangan sampah.

## Business Understanding

### Problem Statements

- Bagaimana memberikan rekomendasi pengelolaan sampah yang tepat berdasarkan jenis sampah?
- Bagaimana membangun model machine learning yang dapat memprediksi pengelolaan sampah yang optimal berdasarkan jenisnya?

### Goals

- Memberikan rekomendasi pengelolaan sampah yang tepat untuk jenis sampah yang diberikan.
- Membangun model prediktif menggunakan machine learning yang dapat memberikan rekomendasi pengelolaan secara otomatis dan efisien.

### Solution Statements

- Menggunakan model neural network untuk memprediksi rekomendasi pengelolaan berdasarkan jenis sampah.
- Meningkatkan akurasi model dengan melakukan pengoptimalan hyperparameter.

## Data Understanding

Dataset yang digunakan berisi informasi mengenai jenis sampah dan rekomendasi pengelolaan terkait. Dataset ini dapat diunduh dari sumber internal atau dibuat berdasarkan pengetahuan yang ada.

### Variabel pada dataset:
- **Jenis_Sampah**: Jenis sampah (misal: Kardus, Plastik, Kertas, dll.)
- **Rekomendasi**: Rekomendasi pengelolaan untuk setiap jenis sampah (misal: Daur ulang, Komposkan, dll.)
- **Edukasi**: Penjelasan mengenai manfaat pengelolaan sampah yang tepat.
- **Dampak_Lingkungan**: Dampak negatif dari pengelolaan yang salah.
- **Proses_Pengolahan**: Cara-cara pengolahan atau daur ulang sampah tersebut.

## Data Preparation

Dalam tahap ini, data yang kita gunakan perlu dipersiapkan agar bisa digunakan oleh model machine learning.

In [47]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
# Dataset
data = {
    'Jenis_Sampah': ['Cardboard', 'Food Organics', 'Glass', 'Metal',
                     'Miscellaneous Trash', 'Paper', 'Plastic',
                     'Textile Trash', 'Vegetation'],
    'Rekomendasi': ['Daur ulang', 'Komposkan', 'Daur ulang', 'Jual ke pengepul',
                    'Buang ke TPS B3', 'Daur ulang', 'Daur ulang', 'Daur ulang', 'Komposkan'],
    'Edukasi': [
        'Daur ulang kardus membantu mengurangi sampah dan menghemat sumber daya alam.',
        'Sampah organik dapat dijadikan pupuk kompos untuk pertanian.',
        'Daur ulang kaca menghemat energi dan mengurangi polusi.',
        'Logam bekas bisa dijual ke pengepul dan didaur ulang.',
        'Sampah misc. harus dibuang sesuai dengan jenisnya ke tempat sampah yang tepat.',
        'Daur ulang kertas mengurangi penggunaan pohon dan membantu kelestarian lingkungan.',
        'Daur ulang plastik membantu mengurangi sampah plastik di lingkungan.',
        'Daur ulang tekstil mengurangi limbah dan dapat digunakan untuk pembuatan produk baru.',
        'Komposkan sampah vegetasi untuk menghasilkan pupuk alami.'
    ],
    'Dampak_Lingkungan': [
        'Sampah kardus yang tidak didaur ulang dapat menambah volume sampah di TPA.',
        'Daur ulang sampah organik mengurangi volume sampah di TPA.',
        'Daur ulang kaca mengurangi polusi dan konsumsi energi dalam produksi kaca baru.',
        'Daur ulang logam mengurangi penggunaan sumber daya alam dan mengurangi polusi.',
        'Sampah yang tidak dikelola dengan baik dapat mencemari tanah dan air.',
        'Daur ulang kertas mengurangi penebangan pohon dan mengurangi polusi.',
        'Sampah plastik yang tidak didaur ulang dapat mencemari lingkungan dan laut.',
        'Daur ulang tekstil dapat mengurangi limbah dan membantu meminimalisir dampak industri tekstil.',
        'Kompos organik membantu menghasilkan pupuk yang ramah lingkungan.'
    ],
    'Proses_Pengolahan': [
        'Kardus dipilah dan dihancurkan untuk didaur ulang menjadi produk baru.',
        'Sampah organik dikomposkan dan digunakan untuk pertanian.',
        'Kaca diproses melalui daur ulang untuk digunakan kembali dalam produksi kaca baru.',
        'Logam diproses dengan cara dilebur untuk menghasilkan produk baru.',
        'Sampah misc. diproses sesuai dengan jenisnya melalui tempat sampah yang sesuai.',
        'Kertas didaur ulang menjadi kertas baru yang dapat digunakan kembali.',
        'Plastik dipilah, dihancurkan, dan diproses menjadi bahan baku plastik baru.',
        'Tekstil didaur ulang menjadi produk baru seperti tas atau bahan tekstil lainnya.',
        'Sampah vegetasi dikomposkan untuk menghasilkan pupuk organik.'
    ]
}

df = pd.DataFrame(data)

# Encode labels
le_jenis_sampah = LabelEncoder()
le_rekomendasi = LabelEncoder()
df['Jenis_Sampah_Encoded'] = le_jenis_sampah.fit_transform(df['Jenis_Sampah'])
df['Rekomendasi_Encoded'] = le_rekomendasi.fit_transform(df['Rekomendasi'])

# Pisahkan fitur dan target
X = df['Jenis_Sampah_Encoded'].values.reshape(-1, 1)
y = df['Rekomendasi_Encoded'].values

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Modeling

Pada tahap ini, kita membangun model neural network untuk memprediksi rekomendasi pengelolaan berdasarkan jenis sampah. Model ini terdiri dari beberapa layer dengan fungsi aktivasi ReLU dan output softmax untuk klasifikasi multi-kelas.

In [49]:
# Membangun model neural network yang lebih sederhana
model = Sequential()
model.add(Dense(8, input_shape=(1,), activation='relu'))  # Lapisan sederhana
model.add(Dense(len(le_rekomendasi.classes_), activation='softmax'))  # Output layer

# Compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(learning_rate=0.01),  # Learning rate lebih besar
              metrics=['accuracy'])

# Latih model dengan lebih sedikit epoch
history = model.fit(X_train, y_train, epochs=50, batch_size=1, verbose=1, validation_split=0.2)

Epoch 1/50


/home/bayuardiyansyah/anaconda3/envs/py310/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - accuracy: 0.3556 - loss: 1.8113 - val_accuracy: 0.5000 - val_loss: 1.1725
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.3556 - loss: 1.3308 - val_accuracy: 0.5000 - val_loss: 1.3302
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.2722 - loss: 1.5108 - val_accuracy: 0.5000 - val_loss: 1.5073
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.4806 - loss: 1.2049 - val_accuracy: 0.5000 - val_loss: 1.7168
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.2472 - loss: 1.1667 - val_accuracy: 0.5000 - val_loss: 1.9309
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.4139 - loss: 1.1218 - val_accuracy: 0.5000 - val_loss: 2.0677
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5778 - loss: 1.1490 - val_accuracy: 0.0000e+00 - val_loss: 2.2058
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5778 - loss: 1.0073 - val_accuracy: 0.0000e+00 - val_loss: 2.3230
Epo

### valuation

Pada bagian ini, kita akan mengevaluasi hasil model menggunakan metrik akurasi untuk mengukur seberapa baik model kita dalam memprediksi jenis sampah dan memberikan rekomendasi pengelolaan yang tepat.

In [50]:
# Evaluasi model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Akurasi model: {accuracy:.2f}")

Akurasi model: 0.00


### Testing

In [51]:
# Fungsi untuk memprediksi rekomendasi
def predict_rekomendasi(jenis_sampah):
    try:
        jenis_sampah_encoded = le_jenis_sampah.transform([jenis_sampah])[0]
        prediksi = model.predict(np.array([[jenis_sampah_encoded]]), verbose=0)
        rekomendasi_encoded = np.argmax(prediksi)
        rekomendasi = le_rekomendasi.inverse_transform([rekomendasi_encoded])[0]
        
        print(f"Jenis Sampah: {jenis_sampah}")
        print(f"Rekomendasi Pengelolaan: {rekomendasi}")
        sampah_info = df[df['Jenis_Sampah'] == jenis_sampah].iloc[0]
        print(f"Edukasi: {sampah_info['Edukasi']}")
        print(f"Dampak Lingkungan: {sampah_info['Dampak_Lingkungan']}")
        print(f"Proses Pengolahan: {sampah_info['Proses_Pengolahan']}")
        print("-" * 50)
        
    except Exception as e:
        print(f"Error: {e}")
        print(f"Jenis sampah '{jenis_sampah}' tidak dikenali dalam dataset.")
        print("-" * 50)

# Tes prediksi
predict_rekomendasi("Plastic")
predict_rekomendasi("Paper")
predict_rekomendasi("Vegetation")

Jenis Sampah: Plastic
Rekomendasi Pengelolaan: Komposkan
Edukasi: Daur ulang plastik membantu mengurangi sampah plastik di lingkungan.
Dampak Lingkungan: Sampah plastik yang tidak didaur ulang dapat mencemari lingkungan dan laut.
Proses Pengolahan: Plastik dipilah, dihancurkan, dan diproses menjadi bahan baku plastik baru.
--------------------------------------------------
Jenis Sampah: Paper
Rekomendasi Pengelolaan: Daur ulang
Edukasi: Daur ulang kertas mengurangi penggunaan pohon dan membantu kelestarian lingkungan.
Dampak Lingkungan: Daur ulang kertas mengurangi penebangan pohon dan mengurangi polusi.
Proses Pengolahan: Kertas didaur ulang menjadi kertas baru yang dapat digunakan kembali.
--------------------------------------------------
Jenis Sampah: Vegetation
Rekomendasi Pengelolaan: Komposkan
Edukasi: Komposkan sampah vegetasi untuk menghasilkan pupuk alami.
Dampak Lingkungan: Kompos organik membantu menghasilkan pupuk yang ramah lingkungan.
Proses Pengolahan: Sampah vegetasi d

In [53]:
predict_rekomendasi("Textile Trash")

Jenis Sampah: Textile Trash
Rekomendasi Pengelolaan: Komposkan
Edukasi: Daur ulang tekstil mengurangi limbah dan dapat digunakan untuk pembuatan produk baru.
Dampak Lingkungan: Daur ulang tekstil dapat mengurangi limbah dan membantu meminimalisir dampak industri tekstil.
Proses Pengolahan: Tekstil didaur ulang menjadi produk baru seperti tas atau bahan tekstil lainnya.
--------------------------------------------------


## Save Model

In [56]:
import tensorflowjs as tfjs

# Simpan model dalam format Keras (.h5)
model.save('/models/RekomendasiSampah/sampah_model.h5', include_optimizer=False)

# Konversi ke format TensorFlow.js
tfjs.converters.save_keras_model(model, '/models/RekomendasiSampah/tfjs_sampah_model')

ModuleNotFoundError: No module named 'tensorflowjs'